In [1]:
import warnings
from math import sqrt
from os.path import exists
# Initialize Wandb
import wandb
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import pandas as pd

from scipy.stats import wasserstein_distance, entropy
from numpy import concatenate
from scipy import stats
from scipy.stats import pearsonr, spearmanr

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional, Dense, Dropout, LSTM, BatchNormalization
from keras.models import Sequential

from tensorflow.keras.layers import Concatenate, Dense, Embedding, Input, LSTM

2025-09-02 17:12:06.354577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-02 17:12:06.418530: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-02 17:12:06.437128: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-02 17:12:06.554622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-02 17:12:07.761786: W tensorflow/compiler/tf2

In [6]:
ori_data = pd.read_csv("energy_data.csv", sep = ',')
ori_data = ori_data.to_numpy()
fake_data = np.load("ddpm_fake_energy_raw.npy")
print(ori_data.shape, fake_data.shape)
fake_data = fake_data.reshape(-1, 57)

# remove the time-related features, specifically the 11,12,13,14 features
exclude_indices = [52, 53, 54, 55]
include_indices = [i for i in range(ori_data.shape[1]) if i not in exclude_indices]
ori_data = ori_data[:, include_indices]
fake_data = fake_data[:, include_indices]

(1947, 57) (2048, 49, 57)


In [7]:
for i in range(ori_data.shape[1]):
    real_vals = ori_data[:, i]
    fake_vals = fake_data[:, i]

    # KL Divergence (hist-based)
    p_hist, bins = np.histogram(real_vals, bins=100, density=True)
    q_hist, _ = np.histogram(fake_vals, bins=bins, density=True)
    p_hist += 1e-10  # avoid log(0)
    q_hist += 1e-10

    kl = entropy(p_hist, q_hist)

    print(f"Feature {i}: KL = {kl:.4f}")

Feature 0: KL = 0.0104
Feature 1: KL = 0.0473
Feature 2: KL = 0.0414
Feature 3: KL = 0.0578
Feature 4: KL = 0.0217
Feature 5: KL = 0.0163
Feature 6: KL = 0.0386
Feature 7: KL = 0.0264
Feature 8: KL = 0.0466
Feature 9: KL = 0.0711
Feature 10: KL = 0.0091
Feature 11: KL = 0.0347
Feature 12: KL = 0.0192
Feature 13: KL = 0.1526
Feature 14: KL = 0.1067
Feature 15: KL = 0.0377
Feature 16: KL = 0.1301
Feature 17: KL = 0.1509
Feature 18: KL = 0.1613
Feature 19: KL = 0.7078
Feature 20: KL = 0.1522
Feature 21: KL = 0.0218
Feature 22: KL = 0.0171
Feature 23: KL = 0.0350
Feature 24: KL = 0.0159
Feature 25: KL = 0.2389
Feature 26: KL = 0.6669
Feature 27: KL = 0.1584
Feature 28: KL = 0.2113
Feature 29: KL = 0.0243
Feature 30: KL = 0.0343
Feature 31: KL = 0.0257
Feature 32: KL = 0.0160
Feature 33: KL = 0.0140
Feature 34: KL = 0.0207
Feature 35: KL = 0.0182
Feature 36: KL = 0.0806
Feature 37: KL = 0.0208
Feature 38: KL = 0.0319
Feature 39: KL = 0.0121
Feature 40: KL = 0.0050
Feature 41: KL = 0.0333
Fe

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
real_scaled = scaler.fit_transform(ori_data)
fake_scaled = scaler.transform(fake_data)

In [9]:
for i in range(real_scaled.shape[1]):
    real_vals = real_scaled[:, i]
    fake_vals = fake_scaled[:, i]

    # Wasserstein
    w_dist = wasserstein_distance(real_vals, fake_vals)

    print(f"Feature {i}: Wasserstein = {w_dist:.4f}")

Feature 0: Wasserstein = 0.0009
Feature 1: Wasserstein = 0.0076
Feature 2: Wasserstein = 0.0016
Feature 3: Wasserstein = 0.0088
Feature 4: Wasserstein = 0.0063
Feature 5: Wasserstein = 0.0032
Feature 6: Wasserstein = 0.0032
Feature 7: Wasserstein = 0.0021
Feature 8: Wasserstein = 0.0030
Feature 9: Wasserstein = 0.0181
Feature 10: Wasserstein = 0.0035
Feature 11: Wasserstein = 0.0053
Feature 12: Wasserstein = 0.0008
Feature 13: Wasserstein = 0.0026
Feature 14: Wasserstein = 0.0060
Feature 15: Wasserstein = 0.0075
Feature 16: Wasserstein = 0.0107
Feature 17: Wasserstein = 0.0048
Feature 18: Wasserstein = 0.0057
Feature 19: Wasserstein = 0.0111
Feature 20: Wasserstein = 0.0134
Feature 21: Wasserstein = 0.0036
Feature 22: Wasserstein = 0.0122
Feature 23: Wasserstein = 0.0022
Feature 24: Wasserstein = 0.0018
Feature 25: Wasserstein = 0.0026
Feature 26: Wasserstein = 0.0039
Feature 27: Wasserstein = 0.0222
Feature 28: Wasserstein = 0.0191
Feature 29: Wasserstein = 0.0166
Feature 30: Wasserst